In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams.update({'figure.max_open_warning': 0})
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
toys = pd.read_csv('./amazon_co-ecommerce_sample.csv', engine='python')

In [3]:
toys.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [4]:
toys.shape

(10000, 17)

In [5]:
toys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   price                                        8565 non-null   object 
 4   number_available_in_stock                    7500 non-null   object 
 5   number_of_reviews                            9982 non-null   object 
 6   number_of_answered_questions                 9235 non-null   float64
 7   average_review_rating                        9982 non-null   object 
 8   amazon_category_and_sub_category             9310 non-null   object 
 9   customers_who_bought_this_item_also_bought   8938 non-null   object 
 10 

In [6]:
toys.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   price                                        8565 non-null   object 
 4   number_available_in_stock                    7500 non-null   object 
 5   number_of_reviews                            9982 non-null   object 
 6   number_of_answered_questions                 9235 non-null   float64
 7   average_review_rating                        9982 non-null   object 
 8   amazon_category_and_sub_category             9310 non-null   object 
 9   customers_who_bought_this_item_also_bought   8938 non-null   object 
 10 

In [7]:
toys.drop(['customers_who_bought_this_item_also_bought','items_customers_buy_after_viewing_this_item','customer_questions_and_answers','customer_reviews','sellers','product_description'], axis=1, inplace=True)
toys.dropna(subset=['price', 'average_review_rating', 'amazon_category_and_sub_category'], inplace=True)

In [8]:
def process_price(price_str):
    # Если значение является числом, то просто возвращаем его
    if isinstance(price_str, float):
        return price_str

    # Заменяем символы, которые могут помешать преобразованию в число
    price_str = price_str.replace('£', '').replace(',', '').strip()

    # Если строка содержит диапазон цен, то возвращаем среднее значение
    if '-' in price_str:
        min_price, max_price = price_str.split('-')
        return (float(min_price.strip()) + float(max_price.strip())) / 2
    else:
        return float(price_str)

# Применяем функцию к столбцу price
toys['price'] = toys['price'].apply(process_price)

In [9]:
toys['average_review_rating'] = toys['average_review_rating'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

In [10]:
toys['number_available_in_stock'].fillna('0', inplace=True)
toys['number_available_in_stock'] = toys['number_available_in_stock'].str.extract(r'(\d+)').astype(int)

/var/folders/9q/sk_9949x0gx4j154w_3t50ww0000gn/T/ipykernel_24340/4066185440.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  toys['number_available_in_stock'].fillna('0', inplace=True)


In [11]:
toys['number_of_reviews'] = toys['number_of_reviews'].apply(lambda x: re.sub(r'\D', '', x)).astype(int)

In [12]:
toys['number_of_answered_questions'].fillna(0, inplace=True)
toys['number_of_answered_questions'] = toys['number_of_answered_questions'].astype(int)

/var/folders/9q/sk_9949x0gx4j154w_3t50ww0000gn/T/ipykernel_24340/2650461214.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  toys['number_of_answered_questions'].fillna(0, inplace=True)


In [13]:
toys.isna().sum()

uniq_id                               0
product_name                          0
manufacturer                          3
price                                 0
number_available_in_stock             0
number_of_reviews                     0
number_of_answered_questions          0
average_review_rating                 0
amazon_category_and_sub_category      0
description                         526
product_information                  52
dtype: int64

In [14]:
toys['manufacturer'].fillna('not specified', inplace=True)
toys['description'].fillna('not specified', inplace=True)
toys['product_information'].fillna('not specified', inplace=True)

/var/folders/9q/sk_9949x0gx4j154w_3t50ww0000gn/T/ipykernel_24340/3686858813.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  toys['manufacturer'].fillna('not specified', inplace=True)
/var/folders/9q/sk_9949x0gx4j154w_3t50ww0000gn/T/ipykernel_24340/3686858813.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

In [15]:
toys.drop(['uniq_id'], axis=1, inplace=True)
toys.reset_index(drop=True, inplace=True)
toys['id'] = toys.index + 1

In [16]:
toys.duplicated().sum()

0

In [17]:
toys.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8003 entries, 0 to 8002
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   product_name                      8003 non-null   object 
 1   manufacturer                      8003 non-null   object 
 2   price                             8003 non-null   float64
 3   number_available_in_stock         8003 non-null   int64  
 4   number_of_reviews                 8003 non-null   int64  
 5   number_of_answered_questions      8003 non-null   int64  
 6   average_review_rating             8003 non-null   float64
 7   amazon_category_and_sub_category  8003 non-null   object 
 8   description                       8003 non-null   object 
 9   product_information               8003 non-null   object 
 10  id                                8003 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 20.0 MB


In [18]:
for col in toys.select_dtypes(include=['float64']).columns:
    toys[col] = pd.to_numeric(toys[col], downcast='float')

In [19]:
for col in toys.select_dtypes(include=['int64']).columns:
    toys[col] = pd.to_numeric(toys[col], downcast='integer')

In [20]:
for col in toys.select_dtypes(include=['object']).columns:
    if len(toys[col].unique()) / len(toys[col]) < 0.5:
        toys[col] = toys[col].astype('category')

In [21]:
toys.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8003 entries, 0 to 8002
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   product_name                      8003 non-null   object  
 1   manufacturer                      8003 non-null   category
 2   price                             8003 non-null   float32 
 3   number_available_in_stock         8003 non-null   int8    
 4   number_of_reviews                 8003 non-null   int16   
 5   number_of_answered_questions      8003 non-null   int8    
 6   average_review_rating             8003 non-null   float32 
 7   amazon_category_and_sub_category  8003 non-null   category
 8   description                       8003 non-null   object  
 9   product_information               8003 non-null   object  
 10  id                                8003 non-null   int16   
dtypes: category(2), float32(2), int16(2), int8(2), object(3)

In [22]:
text_fields = ['product_name', 'description', 'product_information', 'manufacturer', 'amazon_category_and_sub_category']
toys['combined'] = toys[text_fields].apply(lambda row: ' '.join(row), axis=1)

# Создание CountVectorizer для преобразования текста в мешок слов
cv = CountVectorizer(encoding='utf-8')
vectors = cv.fit_transform(toys['combined'].values.astype('U'))

# Вычисление матрицы схожести с использованием косинусного сходства
similarity = cosine_similarity(vectors)

In [23]:
similarity

array([[1.        , 0.48243359, 0.54856754, ..., 0.45070801, 0.41556297,
        0.559738  ],
       [0.48243359, 1.        , 0.5723141 , ..., 0.33061386, 0.311627  ,
        0.34024588],
       [0.54856754, 0.5723141 , 1.        , ..., 0.38454966, 0.39441596,
        0.42873245],
       ...,
       [0.45070801, 0.33061386, 0.38454966, ..., 1.        , 0.47601011,
        0.54487345],
       [0.41556297, 0.311627  , 0.39441596, ..., 0.47601011, 1.        ,
        0.45081074],
       [0.559738  , 0.34024588, 0.42873245, ..., 0.54487345, 0.45081074,
        1.        ]])

In [24]:
def recommend(toy, weight_similarity=0.8, weight_price=0.1, weight_reviews=0.1):
    if toy in toys['product_name'].values:
        toy_index = toys[toys['product_name'] == toy].index[0]
        distances = similarity[toy_index]

        # Invert distances to similarity scores and normalize them
        max_similarity = max(distances)
        normalized_similarity = [d / max_similarity for d in distances]

        # Calculate scaled scores for price and number of reviews
        max_price = toys['price'].max()
        scaled_prices = 1 - (toys['price'] / max_price).values  # Convert Series to numpy array

        max_reviews = toys['number_of_reviews'].max()
        scaled_reviews = (toys['number_of_reviews'] / max_reviews).values  # Convert Series to numpy array

        # Combine similarity scores and feature scores with weights
        if len(normalized_similarity) == len(scaled_prices) == len(scaled_reviews):
            combined_scores = (weight_similarity * np.array(normalized_similarity) +
                               weight_price * np.array(scaled_prices) +
                               weight_reviews * np.array(scaled_reviews))
        else:
            return "Lengths of arrays are not consistent"

        # Find indices of top similar toys
        toy_list = sorted(list(enumerate(combined_scores)), reverse=True, key=lambda x: x[1])[0:15]

        return toy_list
    else:
        return "Toy not found in the dataset"


In [30]:
recommend('Trumpeter 1:35 - Type 82 JASDF 6x6 Command Post')

[(2806, 0.8988013582923227),
 (4660, 0.6988115122216302),
 (2824, 0.6980877584706902),
 (2832, 0.6836348292010891),
 (2745, 0.6763368223396152),
 (2801, 0.675488669881356),
 (2800, 0.6729440736604646),
 (2804, 0.6718373042439417),
 (2739, 0.6714224496278942),
 (2792, 0.6683866907655825),
 (2736, 0.6665876747336351),
 (2760, 0.6661193170182408),
 (2721, 0.6607140441866955),
 (2819, 0.6591916627464873),
 (2766, 0.656663787445663)]

In [26]:
toys.to_csv('./toys.csv', index=False)